In [623]:
import json
from matplotlib import pyplot as plt
import pandas as pd
import os
import glob
import numpy as np
from empiricaldist import Cdf

base_path = '../simulator/bin/simData'

In [624]:
!cd /Users/dzakyzf/lain/iotlab/simulator/bin; python3.7 compute_kpis.py --test=1

38534.70s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


simData

generating KPIs for simData/ori_2_exp3_random3_runs1_motes10_minutes60_disprio1_autoeb3_autok1_autot1_ql1/exec_numMotes_10.dat
KPIs saved in simData/ori_2_exp3_random3_runs1_motes10_minutes60_disprio1_autoeb3_autok1_autot1_ql1/exec_numMotes_10.dat.json

generating KPIs for simData/ori_1_exp3_random3_runs1_motes10_minutes60_disprio0_autoeb0_autok0_autot0_ql0/exec_numMotes_10.dat
KPIs saved in simData/ori_1_exp3_random3_runs1_motes10_minutes60_disprio0_autoeb0_autok0_autot0_ql0/exec_numMotes_10.dat.json
simData/ori_2_exp3_random3_runs1_motes10_minutes60_disprio1_autoeb3_autok1_autot1_ql1/exec_numMotes_10.dat.json
simData/ori_1_exp3_random3_runs1_motes10_minutes60_disprio0_autoeb0_autok0_autot0_ql0/exec_numMotes_10.dat.json


# Rank Result

In [625]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

df = pd.read_csv(f'{base_path}/comparison_merged.csv', sep='\t')
df.sort_values(by='rpljoin_time_m-mean-mean', inplace=True, ignore_index=True)
df = df.style.highlight_min(color = 'orange', axis = 0)
df

,method,trickle_pfailed-mean-mean,trickle_pbusy-mean-mean,current-consumed-mean-mean,rpljoin_time_m-mean-mean,last_count_dio-sum-mean,last_failed_dio-sum-mean,dead_nodes-mean,alive_nodes-mean
0,ori_2_exp3_random3_runs1_motes10_minutes60_disprio1_autoeb3_autok1_autot1_ql1,0.169692,0.414870,0.047096,4.242000,107.000000,6.000000,0.000000,9.000000
1,ori_1_exp3_random3_runs1_motes10_minutes60_disprio0_autoeb0_autok0_autot0_ql0,0.218197,0.450363,0.054021,5.659741,110.000000,10.000000,0.000000,9.000000


# Convergence

In [626]:
metric_to_measure = ['pbusy', 'ptransmit', 'preset', 'pfailed']
metric_to_measure_slotframe = ['pbusy', 'ptransmit', 'preset', 'psent', 'eb_prob', "mbr", "nbr"]

subfolders = list(
    [os.path.join(base_path, x) for x in os.listdir(base_path)]
)

main_result = {}

for subfolder in subfolders:
    if not os.path.isdir(subfolder):
        continue

    mthd = subfolder.split('/')[-1]
    for infile in glob.glob(os.path.join(subfolder, '*.dat.json')):
        with open(infile, 'r') as f:
            data = json.load(f)
            # print(infile)

            result = {}
            for k in metric_to_measure:
                new_key = f'trickle_{k}'
                result[new_key] = {}

            for k in metric_to_measure_slotframe:
                new_key = f'slotframe_{k}'
                result[new_key] = {}

            per_slotframe_run = 0
            for run_id in data.keys():
                for mote_id in data[run_id].keys():
                    if mote_id == 'global-stats':
                        per_slotframe_run = data[run_id]['global-stats']['per_slotframe_run'] 
                        result['per_slotframe_run'] = per_slotframe_run
                        continue
                
                    for it in data[run_id][mote_id]['trickle']:
                        trickle_it = int(it)
                        trickle_data = data[run_id][mote_id]['trickle'][it]

                        for k in metric_to_measure:
                            new_key = f'trickle_{k}'
                            if trickle_it not in result[new_key].keys():
                                result[new_key][trickle_it] = []
                            
                            if k in trickle_data.keys():
                                new_key = f'trickle_{k}'
                                val = trickle_data[k]
                                if val is not None:
                                    result[new_key][trickle_it].append(val)

                    for k in metric_to_measure_slotframe:
                        new_key = f'slotframe_{k}'
                        values = data[run_id][mote_id].get(f'per_slotframe_{k}', [])

                        for idx, val in enumerate(values):
                            if idx not in result[new_key].keys():
                                result[new_key][idx] = []
                            result[new_key][idx].append(val)

            t = infile.split("/")
            algo = t[t.index('simData')+1]
            main_result[algo] = result

### Figure per algorithm

In [627]:
conv_base_path = 'conv_figures/Figure per algorithm'
files = glob.glob(f'{conv_base_path}/*')
for items in files: os.remove(items)

# Trickle states
for algo in main_result:
    fig = plt.figure(figsize=(15,5), dpi=300)
    
    for k in metric_to_measure:
        metric = f'trickle_{k}'

        data = []
        is_over = False
        for state in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][state]
            mean_ = np.mean(values)
            if np.max(values) > 1: is_over = True
            data.append(mean_)
        
        if not is_over: plt.plot(data, label=metric, linestyle='--', marker='o')
        
    plt.legend()
    plt.title(f'Trickle of {algo}')
    plt.xlabel('Trickle states')
    plt.ylabel('Values')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/Trickle_{algo}.png")
    plt.close()

# Per slotframe
for algo in main_result:
    fig = plt.figure(figsize=(15,5), dpi=300)
    per_slotframe_run = main_result[algo]['per_slotframe_run']
    
    for k in metric_to_measure_slotframe:
        metric = f'slotframe_{k}'

        data = []
        is_over = False
        for slotframe in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][slotframe]
            mean_ = np.mean(values)
            if np.max(values) > 1: is_over = True
            data.append(mean_)
        
        if not is_over:
            range_ = [x*per_slotframe_run for x in range(1, len(data)+1)]
            plt.plot(range_, data, label=metric, linestyle='--', marker='o')
    
    plt.legend()
    plt.title(f'Slotframe of {algo}')
    plt.xlabel('Slotframe')
    plt.ylabel('Values')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/Slotframe_{algo}.png")
    plt.close()

### Figure per metric

In [628]:
conv_base_path = 'conv_figures/Figure per metric'
files = glob.glob(f'{conv_base_path}/*')
for items in files: os.remove(items)

# Trickle states
for k in metric_to_measure:
    fig = plt.figure(figsize=(15,5), dpi=300)
    metric = f'trickle_{k}'

    for algo in main_result:

        data = []
        for state in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][state]
            mean_ = np.mean(values)
            data.append(mean_)
        
        plt.plot(data, label=f'{metric}_{algo}', linestyle='--', marker='o')
        
    plt.legend()
    plt.title(f'Trickle of {metric}')
    plt.xlabel('Trickle states')
    plt.ylabel('Values')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/metric_{metric}.png")
    plt.close()

# Per slotframe
for k in metric_to_measure_slotframe:
    fig = plt.figure(figsize=(15,5), dpi=300)
    metric = f'slotframe_{k}'
    
    for algo in main_result:
        data = []
        per_slotframe_run = main_result[algo]['per_slotframe_run']
        for slotframe in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][slotframe]
            mean_ = np.mean(values)
            data.append(mean_)
        
        range_ = [x*per_slotframe_run for x in range(1, len(data)+1)]
        plt.plot(range_, data, label=f'{metric}_{algo}', linestyle='--', marker='o')
    
    plt.legend()
    plt.title(f'Slotframe of {metric}')
    plt.xlabel('Slotframe')
    plt.ylabel('Values')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/metric_{metric}.png")
    plt.close()

In [629]:
# algo = 'ori_5_exp3_random3_runs1_motes10_minutes60_disprio1_autoeb3_autok1_autot1_ql1'
# metric = 'slotframe_nbr'


# data = []
# for slotframe in sorted(main_result[algo][metric].keys()):
#     values = main_result[algo][metric][slotframe]
#     mean_ = int(np.ceil(np.mean(values)))
#     data.append(mean_)

In [630]:
# algo = 'ori_2_exp3_random3_runs1_motes10_minutes60_disprio1_autoeb3_autok1_autot1_ql1'
# metric = 'trickle_pbusy'

# data = []
# for state in sorted(main_result[algo][metric].keys()):
#     values = main_result[algo][metric][state]
#     mean_ = np.mean(values)
#     data.append(mean_)

In [631]:
# temp = np.array(data) + data[-1]
# aa = np.ceil(np.arange(5, 2, -0.1))
# data2 = [*data, *aa, *temp]
# # 
# eb_prob = 1/4
# max_ = 1 - eb_prob

In [632]:
# total = 0
# avgs = []
# avg = 0
# prev_val = None
# move = []
# move2 = []

# for idx, val in enumerate(data2):
#     total += val
#     avg = total / (idx+1)
#     avgs.append(avg)

#     m = 1
#     move.append(m)
#     eb_used_prob = eb_prob
#     eb_used_prob += (max_ / pow(2, val))
    
#     move2.append(eb_used_prob)
#     prev_val = val

# plt.plot(move2)
# plt.plot(len(move2) * [eb_prob])
# plt.show()
# plt.close()

In [633]:
# plt.plot(data2)
# plt.plot(avgs)
# plt.plot(move)
# plt.show()
# plt.close()

### CDF all algorithms

In [634]:
conv_base_path = 'conv_figures/CDF all algorithms'
files = glob.glob(f'{conv_base_path}/*')
for items in files: os.remove(items)

# Trickle states
for k in metric_to_measure:
    fig = plt.figure(figsize=(10,10), dpi=300)
    metric = f'trickle_{k}'

    for algo in main_result:
        data = []
        for state in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][state]
            mean_ = np.mean(values)
            data.append(mean_)
        
        Cdf.from_seq(data).plot()
        
    plt.legend(main_result.keys())
    plt.title(metric)
    plt.xlabel('Values')
    plt.ylabel('CDF')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/{metric}_cdf.png")
    plt.close()


# Per slotframe
for k in metric_to_measure_slotframe:
    fig = plt.figure(figsize=(10,10), dpi=300)
    metric = f'slotframe_{k}'
    
    for algo in main_result:
        data = []
        per_slotframe_run = main_result[algo]['per_slotframe_run']
        for slotframe in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][slotframe]
            mean_ = np.mean(values)
            data.append(mean_)

        Cdf.from_seq(data).plot()
    
    plt.legend(main_result.keys())
    plt.title(metric)
    plt.xlabel('Values')
    plt.ylabel('CDF')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/{metric}_cdf.png")
    plt.close()